In [1]:
!pip install datasets peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 11.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer

model_name = "EMBEDDIA/crosloengual-bert"

In [3]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_COPA")

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/46.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/329k [00:00<?, ?B/s]

In [5]:
CONTEXT_COL = "premise"
QUESTION_COL = "question"
CHOICE_1_COL = "choice1"
CHOICE_2_COL = "choice2"


def preprocess_function(examples):
    """
    The preprocessing function needs to:
    1. Make two copies of the CONTEXT_COL field and combine each of them with QUESTION_COL to recreate how a sentence starts.
    2. Combine QUESTION_COL with each of the two possible choices.
    3. Flatten these two lists so you can tokenize them, and then unflatten them afterward so each example has a corresponding input_ids, attention_mask, and labels field.
    """

    question_headers = examples[QUESTION_COL]

    # Repeat each premise two times to go with the two choice possibilities.
    first_sentences = [[context] * 2 for context in examples[CONTEXT_COL]]
    # Grab all choices possible for each context.
    second_sentences = [
        [f"{header} {examples[choice][i]}" for choice in [CHOICE_1_COL, CHOICE_2_COL]] for i, header
        in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)

    return {k: [v[i: i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

In [6]:

tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'question', 'choice1', 'choice2'], batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [8]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [9]:
from transformers import set_seed

set_seed(42)

model = AutoModelForMultipleChoice.from_pretrained(model_name)
model.config.use_cache = False

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at EMBEDDIA/crosloengual-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import TrainingArguments

new_model_name = "fine_tuned_copa_croslo"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-3,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [12]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1
50,0.705100,0.693147,0.530000,0.528135
100,0.715000,0.693147,0.550000,0.390323
150,0.726000,0.693147,0.550000,0.390323
200,0.697500,0.693147,0.550000,0.390323
250,0.738900,0.693147,0.600000,0.534726
300,0.727100,0.693147,0.550000,0.390323
350,0.732500,0.693147,0.550000,0.390323
400,0.706000,0.693147,0.550000,0.390323


TrainOutput(global_step=400, training_loss=0.7185231876373291, metrics={'train_runtime': 42.8198, 'train_samples_per_second': 74.732, 'train_steps_per_second': 9.341, 'total_flos': 86924534447232.0, 'train_loss': 0.7185231876373291, 'epoch': 8.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.6931471228599548,
 'eval_accuracy': 0.55,
 'eval_f1': 0.3903225806451613,
 'eval_runtime': 0.1687,
 'eval_samples_per_second': 592.792,
 'eval_steps_per_second': 77.063,
 'epoch': 8.0}

In [16]:
trainer.push_to_hub(new_model_name)

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

events.out.tfevents.1716492676.c32a3a36510c.559.0:   0%|          | 0.00/9.77k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

events.out.tfevents.1716492719.c32a3a36510c.559.1:   0%|          | 0.00/826 [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/fine_tuned_copa_croslo/commit/e3271604e992c0f09bee1b1477de40ad76375dbb', commit_message='fine_tuned_copa_croslo', commit_description='', oid='e3271604e992c0f09bee1b1477de40ad76375dbb', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
# Example
choice1 = "Naveličala sta se prepirov."
choice2 = "Izogibala sta se razgovoru o težavi."
prompt = "Odločila sta se skleniti kompromis."
question = "cause"

In [18]:
# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
set_seed(42)

adapter_name = "anzeo/" + new_model_name

tokenizer = AutoTokenizer.from_pretrained(adapter_name)
model = AutoModelForMultipleChoice.from_pretrained(adapter_name)

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/329k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

In [19]:
inputs = tokenizer([[prompt, f"{question} {choice1}"], [prompt, f"{question} {choice2}"]], return_tensors="pt",
                   padding=True)
labels = torch.tensor(0).unsqueeze(0)

outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)
logits = outputs.logits

In [20]:
# Print prediction
logits.argmax().item()

0

In [21]:
#GPU RAM =2.8 GB